In [ ]:
%%time
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 


from Artha.nlp_extraction import *
import Artha.data_process as dp
# from Artha.neocoinsdb import Neo
from Artha.neodb import Neo
import Artha.crypto_data as crypto
from tqdm import tqdm
import csv
import json
from collections import Counter
import numpy as np
np.set_printoptions(suppress=True)
import pprint
import math
from datetime import datetime
import pickle

pp = pprint.PrettyPrinter(indent=1)

# coins database
neo = Neo("neo4j://localhost:7687", "neo4j", "part2")
neo.session.run('''CREATE CONSTRAINT twitter_username IF NOT EXISTS ON (n:Person)
                ASSERT n.username IS UNIQUE''')
neo.session.run('''CREATE CONSTRAINT coin_name IF NOT EXISTS ON (n:Coin)
                ASSERT n.ticker IS UNIQUE''')

In [ ]:
# neo.session.run('''
#                 Merge (n:Person {username: 'checkra_'})
#                 ON CREATE SET n.id ='1356259499431129092',
#                 n.name = 'Checkra',
#                 n.username = 'checkra_'
#         ''')

In [ ]:
print(len(neo.get_nodes()), len(neo.get_relations()))
# without mentions 711 6958

In [ ]:
neo.clear_db()

In [ ]:
neo.load_coins()
# neo.load_follow_nodes(dp.load_following("checkra_"))

In [ ]:
# add follows between only people I follow, can change by getting rid of second 2 last line for anyone a person follows
follows = []
some_people = ["BTC_JackSparrow", "razoreth", "Nostranomist", "CryptoKaleo", "nebraskangooner", "Rager", "SavageBTC"]
available = [i[1:-4] for i in os.listdir("../data/follows/")]

# generate follow edges for people within people group ^
for user in available:
    fs = dp.load_following(user)
    for i in fs:
        if i[3] in available:
            follows.append(i)

In [ ]:
len(follows)

In [ ]:
%%time
print(len(follows))
neo.load_follow_nodes(follows)
neo.load_follow_relations(follows)

In [ ]:
len(available)

In [ ]:
%%time
all_mentions = []
all_docs = []
for ind, name in tqdm(enumerate(available)):
    docs = run_pipeline(name)
    print(ind, name)
    mentions = get_mention_scores(name, docs)
    all_mentions.extend(mentions)
    all_docs.append(docs)

In [ ]:
with open("all_person_docs.pickle", "wb") as p:
    pickle.dump(all_docs, p)
with open("all_mentions.pickle", "wb") as p:
    pickle.dump(all_mentions, p)
print("done")

In [ ]:
len(all_mentions)

In [ ]:
fff = []
for docs in 

In [ ]:
%%time
neo.load_mentions(all_mentions)

In [ ]:
name = "BTC_JackSparrow"
docs = run_pipeline(name)
    # print(tweets[0]["created_at"])
print(name)
singlementions = get_mention_scores(name, docs)

In [ ]:
for i in singlementions:
    print(i)

In [ ]:
for doc in docs:
    if "WRX" in doc._.tickers:
        print(doc, doc._.tweeted_at)

In [ ]:
all_docs[1][0]._.tweeted_at

In [ ]:
%%time
neo.load_mentions(singlementions)

In [ ]:
result = neo.session.run('''
    CALL gds.graph.create(
        'mentions',
        {Person: {label: "Person"},Coin: {label: "Coin"}},
        {CALLOUT: {type : 'CALLOUT', orientation: 'NATURAL'},
            FOLLOWS: {type: 'FOLLOWS', orientation: 'NATURAL'}
        },
        { relationshipProperties: 'weight'}
    )
    YIELD graphName, nodeCount, relationshipCount
    ''')
[r for r in result]

In [ ]:
result = neo.session.run('''
    CALL gds.pageRank.stream('mentions', 
        {
        maxIterations: 20,
        dampingFactor: 0.85,
        relationshipWeightProperty: 'weight'
        })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).ticker AS ticker, score
    ORDER BY score DESC, ticker ASC

''')
f = [r for r in result]
for i in f:
    if i["score"]>.1500001:
        print(i)

In [ ]:
result = neo.session.run("CALL gds.graph.drop('mentions') YIELD graphName")
[r for r in result]

In [ ]:
# make graphview that only looks at people who mention coins, and coins that are mentioned

# exchange tokens might have more mentions
# factor in mentions of coins in threads, so it gets more mentions than just 1